In [ ]:
from dataManager import getData, getChipData, getMarginData, getStockCodeList
import matplotlib.pyplot as plt
import pandas as pd

# 讀取資料，並合併資料

In [ ]:
getStockCodeList()

In [ ]:
code = 'M2500'
price_data = getData(code)
chip_data = getChipData(code)
margin_data = getMarginData(code)

# 查看資料

In [ ]:
price_data.head()

In [ ]:
chip_data.head()

In [ ]:
margin_data.head()

In [ ]:
price_data.columns

# 資料合併

In [ ]:
price_merge_chip = price_data.merge(chip_data, on=['證券代碼', '簡稱', '年月日'])
price_merge_chip

In [ ]:
target_data = price_merge_chip.merge(margin_data, on=['證券代碼', '簡稱', '年月日'])
target_data

# 樣本內樣本外資料分割

In [ ]:
train_index = int(target_data.shape[0]*0.7)

In [ ]:
training_data = target_data.iloc[:train_index,:]
testing_data = target_data.iloc[train_index:,:]

In [ ]:
print("樣本內－訓練期間 ：{} ~ {} ".format(training_data['年月日'].values[0], training_data['年月日'].values[-1]))
print("樣本外－測試期間 ：{} ~ {} ".format(testing_data['年月日'].values[0], testing_data['年月日'].values[-1]))

# 建立 VAR 模型

In [ ]:
from statsmodels.tsa.api import VAR

In [ ]:
target_data.columns

In [ ]:
variable_list = ['ret', '外資總投資股率%', '投信持股率%', '自營持股率%', '融資增減比率', '融券增減比率']
lag_order = 5


var_model = VAR(training_data[variable_list])
var_model_result = var_model.fit(lag_order)

In [ ]:
var_model_result.summary()

# 樣本外的預測

In [ ]:
last_training_data = training_data.tail(lag_order)[variable_list]

In [ ]:
last_training_data

In [ ]:
forecast_step = 30

prediction_vector = var_model_result.forecast(last_training_data.values, forecast_step)

In [ ]:
var_model_result.forecast?

In [ ]:
prediction_vector[0]

In [ ]:
predict_ret_list = prediction_vector[:, 0]

# 預測的報酬率圖

In [ ]:
fig = plt.figure(figsize=(20, 20))
ax = fig.add_subplot(211)
ax2 = fig.add_subplot(212)

ax.set_title('Predict Ret')
ax.plot(predict_ret_list)
ax.grid()
ax2.set_title('Real Ret')
ax2.plot(testing_data.head(forecast_step)['ret'], c='k')
ax2.grid()
plt.show()

# 樣本外收盤價預測

In [ ]:
last_close_price = training_data['收盤價(元)'].values[-1]
next_pred_ret = predict_ret_list[0]
pred_close = last_close_price * (1 +next_pred_ret)
predict_close_price_list = [pred_close]

In [ ]:
for idx, next_pred_ret in enumerate(predict_ret_list[1:]):
    last_close_price = predict_close_price_list[idx]
    pred_close =  last_close_price * (1 + next_pred_ret)
    predict_close_price_list.append(pred_close)

In [ ]:
fig = plt.figure(figsize=(20, 8))

plt.plot(predict_close_price_list, marker="o", label='Predict')
plt.plot(testing_data['收盤價(元)'].values[: forecast_step], marker="o", c='k', label='Real')

plt.legend()
plt.grid()
plt.show()